In [1]:
%pip install spmf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from spmf import Spmf
import pandas as pd
from itertools import chain, combinations

In [3]:
from itertools import chain, combinations

def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

def associationRule(freqItemSetDict, itemSetList, minConf):
    rules = []

    # Menghitung dukungan setiap itemset dalam freqItemSet
    itemSetSupport = {}
    for itemSet, support in freqItemSetDict:
        itemSetTuple = frozenset(itemSet)  # Menggunakan frozenset sebagai kunci
        itemSetSupport[itemSetTuple] = support

    for itemSet, support in freqItemSetDict:
        subsets = powerset(itemSet)
        itemSetSup = itemSetSupport[frozenset(itemSet)]  # Mengambil dukungan itemSet

        for s in subsets:
            s_tuple = frozenset(s)
            if s_tuple in itemSetSupport and itemSetSupport[s_tuple] > 0:
                confidence = float(itemSetSup / itemSetSupport[s_tuple])
                if confidence >= minConf:
                    support_B = (itemSetSupport[frozenset(itemSet.difference(s))] / len(itemSetList))
                    
                    lift_ratio = confidence / support_B

                    rules.append([
                        set(s),
                        set(itemSet.difference(s)),
                        support / len(itemSetList) * 100,
                        confidence,
                        support_B,
                        lift_ratio
                    ])

    return rules


In [4]:
# file_path = '../notebook/data/example_spfm_test.txt'

# file_path = '../notebook/data/transaction_01_sd_03_october_spmf_test.txt'
# minsup: 0.0014, minconf: 0.6, minsuppcount: 2.1 

# file_path = '../notebook/data/transaction_01_sd_07_october_spmf_test.txt'
# minsup: 0.0008, minconf: 0.6, minsuppcount: 2.7 

# file_path = '../notebook/data/transaction_13_sd_15_october_spmf_test.txt'
# minsup: 0.002, minconf: 0.6, minsuppcount: 2.6

file_path = '../notebook/data/[ECLAT] Validation Test SPMF.txt'
# minsup: 0.0004, minconf: 0.6, minsuppcount: 3.9

itemSetList = []

with open(file_path, 'r') as file:
    for line in file:
        elements = line.strip().split(' ')
        
        if elements[0].startswith('@'):
            continue  # Lewati baris yang mengandung informasi header
        
        itemSetList.append(elements)


spmf = Spmf("Eclat",
            input_filename=file_path,
            output_filename="../notebook/output/frequent-pattern-eclat.txt",
            arguments=[0.0004, False])
spmf.run()

patterns_list = []
with open("../notebook/output/frequent-pattern-eclat.txt", "r") as file:
    for line in file:
        if line.startswith("#SUP:"):
            continue
        parts = line.strip().split(" #SUP: ")
        pattern = frozenset(map(str, parts[0].split()))
        sup = int(parts[1])
        patterns_list.append((pattern, sup))

# Create association rules
rules = associationRule(patterns_list, itemSetList, minConf=0.6)

# Buat DataFrame
columns = ["Antecedent", "Consequent", "Support", "Confidence", "Support B", "Lift"]
df = pd.DataFrame(rules, columns=columns)

# Konversi kolom 'Confidence' ke float
df['Confidence'] = df['Confidence'].astype(float)

# Lakukan pengurutan
df_sorted = df.sort_values(by='Confidence', ascending=False)
df_sorted = df_sorted.reset_index(drop=True)

# Tampilkan DataFrame yang diurutkan
df_sorted

>/D:/dev/project/Skripsi/Skripsi_ECLAT_and_FPGrowth/notebook/spmf.jar
=============  ECLAT v0.96r18 - STATS =============
 Transactions count from database : 9793
 Frequent itemsets count : 1862
 Total time ~ 210 ms
 Maximum memory usage : 42.06133270263672 mb
Post-processing to show result in terms of string values.
Post-processing completed.



,Antecedent,Consequent,Support,Confidence,Support B,Lift
0,{8991001111319},{8991001111296},0.040846,1.000000,0.000613,1632.166667
1,{8999999529628},{GLLMP1KG},0.040846,1.000000,0.014704,68.006944
2,"{GLLMP1KG, 089686010107}",{089686010824},0.051057,1.000000,0.021954,45.548837
3,"{8995177101112, 089686010107}",{089686010824},0.040846,0.800000,0.021954,36.439070
4,"{8992717781025, 089686010107}",{089686010824},0.040846,0.800000,0.021954,36.439070
5,"{8992696423084, 089686010107}",{8886008101053},0.040846,0.800000,0.055754,14.348718
6,"{089686010824, 711844110113}",{089686010107},0.051057,0.714286,0.021240,33.629808
7,"{8992839007577, 089686010824}",{089686010107},0.051057,0.714286,0.021240,33.629808
8,{8992907953010},{8992907952327},0.071480,0.700000,0.011539,60.664602
9,"{089686010824, 8995177101112}",{089686010107},0.040846,0.666667,0.021240,31.387821
